In [1]:
import sys
from pathlib import Path
from os.path  import join
from src.config import DATA_DIR, CONFIG_DIR
from omegaconf import OmegaConf
from src.external.hptr.src.data_modules.agent_centric import AgentCentricPreProcessing
from src.external.hptr.src.data_modules.ac_global import AgentCentricGlobal
from src.mimolm import InputProjections, EarlyFusionEncoder, MotionDecoder
import torch
import torch.nn.functional as F
import lightning as pl

# Add the project root to sys.path
project_root = Path().resolve().parent  # Adjust as needed to point to the root folder
sys.path.append(str(project_root))

print(Path.cwd())  # Check if the path is added

2025-01-27 18:43:57.252 | INFO     | src.config:<module>:11 - PROJ_ROOT path is: G:\Work\DS\mimolm
c:\Users\Harshavardhan Patil\.virtualenvs\mimolm-kv68Yv56\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


g:\Work\DS\mimolm\notebooks


In [4]:
from src.external.hptr.src.data_modules.data_h5_av2 import DataH5av2
from src.mimolm import MimoLM

data_module = DataH5av2(DATA_DIR / "interim" / "train" )
data_module.setup(stage="fit")
data_loader = data_module.train_dataloader()

model = MimoLM(data_size=data_module.tensor_size_train)

trainer = pl.Trainer(max_epochs=2)
trainer.fit(model=model, train_dataloaders=data_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name              | Type               | Params | Mode 
-----------------------------------------------------------------
0 | preprocessor      | Sequential         | 0      | train
1 | input_projections | InputProjections   | 469 K  | train
2 | encoder           | EarlyFusionEncoder | 2.4 M  | train
3 | decoder           | MotionDecoder      | 1.6 M  | train
4 | logsoftmax        | LogSoftmax         | 0      | train
5 | criterion         | NLLLoss            | 0      | train
-----------------------------------------------------------------
4.5 M     Trainable params
0         Non-trainable params
4.5 M     Total params
18.071    Total estimated model params size (MB)
94        Modules in train mode
0         Modules in eval mode


Input projections parameters: 0.47M
Encoder parameters: 2.42M
Decoder parameters: 1.63M
Epoch 1: 100%|██████████| 50/50 [03:30<00:00,  0.24it/s, v_num=4]

`Trainer.fit` stopped: `max_epochs=2` reached.


Epoch 1: 100%|██████████| 50/50 [03:31<00:00,  0.24it/s, v_num=4]


In [ ]:
model.to

In [3]:
hjask

NameError: name 'hjask' is not defined

In [2]:
from src.external.hptr.src.data_modules.data_h5_av2 import DataH5av2

data_module = DataH5av2(DATA_DIR / "interim" / "train" )
data_module.setup(stage="fit")
dl = data_module.train_dataloader()

In [3]:
from src.mimolm import MimoLM

model = MimoLM(data_size=data_module.tensor_size_train)

Input projections parameters: 0.47M
Encoder parameters: 2.42M
Decoder parameters: 1.63M


In [16]:
from src.modeling.modules.lm_utils import tokenize_motion

#dic = OmegaConf.load(Path(CONFIG_DIR / "model/ac_wayformer.yaml"))
episode = None
fmask = None
prepreprocess = AgentCentricPreProcessing(time_step_current=49, 
                                          data_size=data_module.tensor_size_train, 
                                          n_target=8,
                                          n_other=48,
                                          n_map=512,
                                          mask_invalid=False)

preprocess = AgentCentricGlobal(time_step_current=49,
                                data_size=data_module.tensor_size_train,
                                dropout_p_history=0.15, 
                                add_ohe=True,
                                pl_aggr=False,
                                pose_pe= {"agent": "xy_dir",
                                        "map": "mpa_pl"})

inputproj = InputProjections(hidden_dim=256,
                             agent_attr_dim=preprocess.model_kwargs["agent_attr_dim"],
                             map_attr_dim=preprocess.model_kwargs["map_attr_dim"],
                             n_step_hist=preprocess.model_kwargs["n_step_hist"],
                             n_pl_node=preprocess.model_kwargs["n_pl_node"],
                             add_learned_pe=True,
                             n_layer_mlp=3,
                             mlp_cfg={"end_layer_activation" : True,
                                        "use_layernorm" :  False,
                                        "use_batchnorm" : False,
                                        "dropout_p" : None,})

encoder = EarlyFusionEncoder(hidden_dim=256,
                             tf_cfg={"n_head": 4,
                                     "dropout_p": 0.1,
                                     "norm_first": True,
                                     "bias": True},
                             latent_query={"use_agent_type": False,
                                           "mode_emb": "none", # linear, mlp, add, none
                                           "mode_init": "xavier", # uniform, xavier
                                           "scale": 5.0},
                             n_latent_query=192,
                             n_encoder_layers=2)  

decoder = MotionDecoder()
logsoftmax = torch.nn.LogSoftmax(dim=-1)
loss_fn = torch.nn.NLLLoss()

In [19]:
f = None
femb = None
a = None
l = None
for batch in dl:
    batch = prepreprocess(batch=batch)
    batch = preprocess(batch=batch)
    actuals = tokenize_motion(batch["gt/pos"],
                            decoder.pos_bins, 
                            decoder.verlet_wrapper, 
                            decoder.n_verlet_steps, 
                            decoder.n_time_steps)
    a = actuals
    motion_tokens = torch.cat((batch["ac/target_pos"], batch["gt/pos"]), dim = -2)
    target_types = batch["ac/target_type"]
    input_dict = {
    k.split("input/")[-1]: v for k, v in batch.items() if "input/" in k
    }
    valid = input_dict["target_valid"].any(-1)
    target_emb, target_valid, other_emb, other_valid, map_emb, map_valid = inputproj(target_valid = input_dict["target_valid"], 
              target_attr = input_dict["target_attr"],
              other_valid = input_dict["other_valid"],
              other_attr = input_dict["other_attr"],
              map_valid = input_dict["map_valid"],
              map_attr = input_dict["map_attr"],)
    fused_emb, fused_emb_invalid = encoder(
                target_emb, target_valid, other_emb, other_valid, map_emb, map_valid, input_dict["target_type"], valid
            )
    pred = decoder(motion_tokens, target_types, fused_emb, fused_emb_invalid)
    episode = pred
    loss = loss_fn(
        logsoftmax(pred[:, 50:, :].flatten(0, 1)), 
        actuals.flatten(0, 1).flatten(0, 1).repeat(64))
    print(loss)
    break

torch.Size([2, 8, 110])
tensor(5.1317, grad_fn=<NllLossBackward0>)


In [5]:
episode.shape

torch.Size([1024, 110, 169])

In [6]:
episode[:, 50:, :].flatten(0, 1).shape

torch.Size([61440, 169])

In [ ]:
loss_fn(F.log_softmax(episode[:, 50:, :].flatten(0, 1), dim=-1), a.flatten(0, 1).flatten(0, 1))

In [9]:
a.shape

torch.Size([2, 8, 60])

In [14]:
a.flatten(0, 1).flatten(0, 1).shape

torch.Size([960])

In [7]:
femb.shape

torch.Size([16, 192, 256])

In [8]:
temp = torch.arange(0, 110)
temp.unsqueeze(0).repeat(motion_tokens.shape[1], 1).unsqueeze(0).repeat(motion_tokens.shape[0], 1, 1)

tensor([[[  0,   1,   2,  ..., 107, 108, 109],
         [  0,   1,   2,  ..., 107, 108, 109],
         [  0,   1,   2,  ..., 107, 108, 109],
         ...,
         [  0,   1,   2,  ..., 107, 108, 109],
         [  0,   1,   2,  ..., 107, 108, 109],
         [  0,   1,   2,  ..., 107, 108, 109]],

        [[  0,   1,   2,  ..., 107, 108, 109],
         [  0,   1,   2,  ..., 107, 108, 109],
         [  0,   1,   2,  ..., 107, 108, 109],
         ...,
         [  0,   1,   2,  ..., 107, 108, 109],
         [  0,   1,   2,  ..., 107, 108, 109],
         [  0,   1,   2,  ..., 107, 108, 109]]])

In [9]:
episode.shape

torch.Size([1024, 110, 169])

In [ ]:
episode[1] == episode.reshape((-1, 256))[110:220]

In [11]:
fmask.shape

torch.Size([16, 192])

In [12]:
fused_emb.shape

torch.Size([16, 192, 256])

In [81]:
fmask.shape

torch.Size([6, 192])